In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import matplotlib
import skimage

from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from dataclasses import dataclass

In [6]:
%matplotlib qt

cap_tracking = cv.VideoCapture(r"..\datasets\unzipped\endovis\Tracking_Robotic_Training\Training\Dataset2\Video.avi")
cap_segmentation = cv.VideoCapture(r"..\datasets\unzipped\endovis\Segmentation_Robotic_Training\Training\Dataset2\Segmentation.avi")
tracking_data = open(r"..\datasets\unzipped\endovis\Tracking_Robotic_Training\Training\Dataset2\Pose.txt", "r")

stop = False
    
frame_number = 0

pred = []
actual = []

while True:
    ret_seg, frame_seg = cap_segmentation.read()
    if frame_seg is None:
        print("End of video, breaking")
        break

    frame_number += 1

    gs = cv.cvtColor(frame_seg, code = cv.COLOR_BGR2GRAY)
    im = gs

    canny = cv.Canny(gs, 70, 160)
    canny_blur = cv.GaussianBlur(canny, (3, 3), 1.0)

    # calculate moments of binary image
    M_shaft = cv.moments(cv.inRange(gs, 155, 165))
    
    # # calculate x,y coordinate of center
    cX_shaft = int(M_shaft["m10"] / M_shaft["m00"])
    cY_shaft = int(M_shaft["m01"] / M_shaft["m00"])
    
    # # put text and highlight the center
    # cv.circle(frame_tracking, (cX_shaft, cY_shaft), 5, (0, 255, 0), -1)

    # calculate moments of binary image
    M_eef = cv.moments(cv.inRange(gs, 65, 75))
    
    # # calculate x,y coordinate of center
    cX_eef = int(M_eef["m10"] / M_eef["m00"])
    cY_eef = int(M_eef["m01"] / M_eef["m00"])

    # cv.circle(frame_tracking, (cX_eef, cY_eef), 5, (0, 255, 0), -1)

    # cv.line(frame_tracking, (cX_eef, cY_eef), (cX_shaft, cY_shaft), color = (0, 255, 0))

    x, y = skimage.draw.line(cX_eef, cY_eef, cX_shaft, cY_shaft)
    found = False
    for i, j in zip(x, y):
        if canny_blur[j,i] > 0: 
            found = True
            break
    
    if not found:
        i = j = -1
        print((cX_eef, cY_eef), (cX_shaft, cY_shaft))
        cv.line(canny_blur, (cX_eef, cY_eef), (cX_shaft, cY_shaft), color = 255)
        plt.imshow(canny_blur)
        
        cv.waitKey(0)

    vec = np.array([cX_shaft - cX_eef, -(cY_shaft - cY_eef)], dtype = np.float64)
    vec /= np.linalg.norm(vec)
    pred.append((i,j,-vec[0],vec[1]))
    
    vals = tracking_data.readline().split(" ")[:4]
    actual.append((int(vals[0]), int(vals[1]), float(vals[2]), float(vals[3])))

    if frame_number % 100 == 0:
        print(frame_number)

    # cv.circle(frame_tracking, (i, j), 5, (255, 0, 0), -1)
    # # 573 346
    # cv.circle(frame_tracking, (521, 289), 5, (255, 255, 0), -1)

    # put text and highlight the center



    # # display the image
    # cv.imshow("Image", frame_tracking)
    # cv.waitKey(0)

# plt.imshow(grad, cmap = "gray")
# plt.colorbar()
# # cv.imshow('Frame', np.concatenate((value, masked_img_gray[:,:,0]), axis = 1))

# frame_number += 1

# while True:
#     # Exit if 'q' is pressed
#     if cv.waitKey(1) & 0xFF == ord('q'):
#         stop = True
#         break
#     elif cv.waitKey(1) == 32: 
#         break
                                                                
# cv.destroyAllWindows()
# cap_segmentation.release()
# cap_tracking.release()

100
200
300
400
500
600
700
800
900
1000
1100
End of video, breaking


In [7]:
plt.figure()
plt.plot([x[0] for x in pred])
plt.plot([x[1] for x in actual])

In [8]:
plt.figure()
plt.plot([np.degrees(np.arctan2(x[3], x[2])) for x in pred])
plt.plot([np.degrees(np.arctan2(x[3], x[2])) for x in actual])

In [5]:
# cap = cv.VideoCapture(r"..\datasets\unzipped\endovis\Tracking_Robotic_Training\Training\Dataset4\Video.avi")

# # Feature parameters
# feature_params = {
#     "maxCorners": 100,
#     "qualityLevel": 0.3,
#     "minDistance": 7,
#     "blockSize": 7 
# }

# # Lucas Kinade parameters
# lk_params = {
#     "winSize": (25, 25),
#     "maxLevel": 4,
#     "criteria": (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
# }

# # Create some random colors
# color = np.random.randint(0,255,(100,3))

# # Take first frame and find corners in it
# ret, old_frame = cap.read()
# old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
# p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# # Create a mask image for drawing purposes
# mask = np.zeros_like(old_frame)

# # Initialize flag to end viewing when q is pressed
# stop = False

# frame_number = 0

# while (not stop):
#     ret,frame = cap.read()
#     if frame is None:
#         print("End of video, breaking")
#         break
    
#     frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

#     # calculate optical flow
#     p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

#     if p1 is None: 
#         print("Failed to extract features")
#         continue

#     # Select good points
#     good_new = p1[st==1]
#     good_old = p0[st==1]
    
#     try:
#         # Draw the tracks
#         for i,(new,old) in enumerate(zip(good_new,good_old)):
#             a,b = new.ravel()
#             c,d = old.ravel()
            
#             # Set values to integers to allow for drawing
#             a = int(a)
#             b = int(b)
#             c = int(c)
#             d = int(d)

#             mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
#             frame = cv.circle(frame, (a,b), 5, color[i].tolist(), -1)
            
#         img = cv.add(frame,mask)
        
#         cv.imshow('Frame', img)
#         # cv.imwrite(os.path.join(PATH, f"vid_{ii}_{frame_number}.png"), img)
        
#         frame_number += 1
        
#         while True:
#             # Exit if 'q' is pressed
#             if cv.waitKey(25) & 0xFF == ord('q'):
#                 stop = True
#                 break
#             elif cv.waitKey(33) == 32: break
                                                                    
#         # Now update the previous frame and previous points
#         old_gray = frame_gray.copy()
#         p0 = good_new.reshape(-1,1,2)
#     except Exception as e:
#         print(f"Iteration failed: {e}")

KeyboardInterrupt: 